# Overfitting e regolarizzazione
L'overfitting è un problema tipico del machine learning che si manifesta quando un modello si lega troppo ai dati di addestramento e fallisce nel generalizzare su dati nuovi.

L'overffiting è caratterizzato da:
* **Alta variaza**: le previsioni per modelli addestrati con diverse parti del dataset saranno molto diverse tra loro.
* **Basso bias**: l'errore per le predizioni sul set di addestramento è mediamente molto basso

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
path = "https://frenzy86.s3.eu-west-2.amazonaws.com/IFAO/boston_houses.csv"
df = pd.read_csv(path)

In [3]:
df.rename(columns={'MEDV':'Price'},inplace=True)

In [4]:
target = 'Price'
X = df.drop(target,axis=1).values
y = df[target].values

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.3,
                                                    random_state=667,
                                                    )

### Creiamo le features polinomiali
Per correggere l'overfitting prima dobbiamo causarlo, un buon modo è aumentare la complessità del nostro modello aumentando il numero di features utilizzando i polinomi.

In [5]:
polyfeats = PolynomialFeatures(degree=2)
X_train_poly = polyfeats.fit_transform(X_train)
X_test_poly = polyfeats.transform(X_test)

print("Numero di esempi nel test: "+str(X_train_poly.shape[0]))
print("Numero di features: "+str(X_train_poly.shape[1]))

Numero di esempi nel test: 354
Numero di features: 105


### Standardizziamo i dati
**NOTA BENE** Per applicare la regolarizzazione è sempre necessario portare i dati sulla stessa scala.

In [6]:
ss = StandardScaler()
X_train_poly = ss.fit_transform(X_train_poly)
X_test_poly = ss.transform(X_test_poly)

Adesso il nostro set di addestramento contiene 354 e 105 features, abbastanza complesso !

### Riconoscere l'overfitting
Evidenziare un problema di overfitting è molto semplice, un modello che ne soffre avrà memorizzato la struttura dei dati di addestramento, piuttosto che imparare da essi, quindi l'errore per le predizioni sul train set sarà molto basso, invece fallirà nel generalizzare, perciò l'errore nel test set sarà decisamente più alto.<br><br>
Quindi per riconoscere l'overfitting è sufficente confrontare questi due valori, scriviamo una funzione che ci permette di farlo in modo da non dover scrivere più volte lo stesso codice.

In [7]:
def overfit_eval(model, X, y):

    """
    model: il nostro modello predittivo già addestrato
    X: una tupla contenente le prorietà del train set e test set (X_train, X_test)
    y: una tupla contenente target del train set e test set (y_train, y_test)
    """

    y_pred_train = model.predict(X[0])
    y_pred_test = model.predict(X[1])

    mse_train = mean_squared_error(y[0], y_pred_train)
    mse_test = mean_squared_error(y[1], y_pred_test)

    r2_train = r2_score(y[0], y_pred_train)
    r2_test = r2_score(y[1], y_pred_test)

    print("Train set:  MSE="+str(mse_train)+" R2="+str(r2_train))
    print("Test set:  MSE="+str(mse_test)+" R2="+str(r2_test))

### Regressione lineare non regolarizzata
Cominciamo eseguendo una regressione lineare (in realtà si tratta di una regressione polinomiale) senza applicare la regolarizzazione.

In [8]:
ll = LinearRegression()
ll.fit(X_train_poly, y_train)

overfit_eval(ll, (X_train_poly, X_test_poly),(y_train, y_test))

Train set:  MSE=15.034347840028595 R2=0.8179241488460587
Test set:  MSE=32.03088973402763 R2=0.6389761364103487


Il modello predice in maniera estremamente (o meglio dire eccessivamente) accurata i dati del train set, mentre è molto più scarso sul test set. Siamo di fronte ad un caso di overfitting.

## Regolarizzazione L2: Ridge Regression
La ridge regression è un modello di regressione lineare che applica la **regolarizzazione L2**, la quale consiste nell'aggiungere una penalità per i pesi nella funzione di costo durante la fase di addestramento.<br>
La penalità è data dalla somma dei quadrati dei pesi:
$$\lambda\sum_{j=1}^{M}W_j^2$$<br>
**Lambda** (conosciuto anche come **alpha**) è il **parametro di regolarizzazione** ed è un'altro iperparametro.
Eseguiamo diverse Ridge regression per diversi valori di alpha.

In [9]:
from sklearn.linear_model import Ridge

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10] #alpha corrispone a lambda

for alpha in alphas:
    print("Alpha="+str(alpha))
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_poly, y_train)

    overfit_eval(ridge, (X_train_poly, X_test_poly),(y_train, y_test))

Alpha=0.0001
Train set:  MSE=3.994688160485263 R2=0.9516216962216072
Test set:  MSE=15.152641036952769 R2=0.8292128299846662
Alpha=0.001
Train set:  MSE=4.008703827485975 R2=0.9514519572661302
Test set:  MSE=15.124700794392753 R2=0.8295277476907432
Alpha=0.01
Train set:  MSE=4.058274133283384 R2=0.9508516282251861
Test set:  MSE=15.105305007020014 R2=0.8297463598539649
Alpha=0.1
Train set:  MSE=4.402406182868336 R2=0.9466839625236713
Test set:  MSE=15.48999674799777 R2=0.8254104547394941
Alpha=1
Train set:  MSE=5.509493608081705 R2=0.9332764048835013
Test set:  MSE=16.523397957976183 R2=0.8137628701559074
Alpha=10
Train set:  MSE=8.278399484203565 R2=0.8997431316398327
Test set:  MSE=20.189297287953412 R2=0.7724440947291633


La Ridge regression, applicando la regolarizzazione L2, ci permette di ridurre l'overfitting e portare l'R2 fino ad un valore di 0.791 per alpha uguale a 10.

## Regolarizzazione L1: Lasso
Lasso è un modello di regressione lineare che applica la regolarizzazione L1, questa funziona in egual modo alla L2, con la differenza che il termine di regolarizza sarà dato dalla somma del valore assoluto dei pesi:
$$\lambda\sum_{j=1}^{M}|W_j|$$<br>
e viene sempre applicato alla funzione di costo durante la fase di addestramento

In [10]:
from sklearn.linear_model import Lasso

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10] #alpha corrisponde a lambda

for alpha in alphas:
    print("Alpha="+str(alpha))
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_poly, y_train)

    overfit_eval(lasso, (X_train_poly, X_test_poly),(y_train, y_test))

Alpha=0.0001
Train set:  MSE=4.879663235273139 R2=0.9409040653867656
Test set:  MSE=17.58861506387127 R2=0.8017566849289173
Alpha=0.001
Train set:  MSE=4.909409855251361 R2=0.9405438142332658
Test set:  MSE=17.42818482600246 R2=0.8035649126988277
Alpha=0.01
Train set:  MSE=5.916971830313482 R2=0.9283415753232851
Test set:  MSE=17.03221744386145 R2=0.8080279068692382
Alpha=0.1
Train set:  MSE=11.021977307534174 R2=0.8665165977917872
Test set:  MSE=25.793165540963034 R2=0.7092822473827975
Alpha=1
Train set:  MSE=18.703798184263565 R2=0.7734846891632877
Test set:  MSE=35.19362698600457 R2=0.603328558971542
Alpha=10
Train set:  MSE=82.57189377254302 R2=0.0
Test set:  MSE=88.72279474960983 R2=-4.883253304388546e-06


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.614e+02, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.387e+02, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.399e+02, tolerance: 2.923e

Lasso ci permette di ottenere un modello ancora migliore, con un R2 di 0.803 per Lambda uguale a 0.1.<br>
Da notare che per valori di lambda più grandi il modello peggiora, questo perché l'effetto della regolarizzazione sarà molto pesante e buona parte dei pesi saranno portati a 0.

## L2 ed L1 insieme: ElasticNet
ElasticNet è un modello di regressione lineare che implementa entrambe le tecniche di regolarizzazone L2 ed L1.<br>
Tramite il parametro <span style="font-family: Monaco">l1_ration</span> possiamo controllare l'effetto delle due regolarizzazione
 * **<span style="font-family: Monaco">l1_ration>0.5</span>** l'effetto della regolarizzazione L1 sarà più intenso rispetto alla L2.
 * **<span style="font-family: Monaco">l1_ration<0.5</span>** l'effetto della regolarizzazione L2 sarà più intenso rispetto alla L1.

In [11]:
from sklearn.linear_model import ElasticNet

alphas = [0.0001, 0.001, 0.01, 0.1 ,1 ,10]

for alpha in alphas:
    print("Lambda is: "+str(alpha))
    elastic = ElasticNet(alpha=alpha, l1_ratio=0.5)
    elastic.fit(X_train_poly, y_train)
    overfit_eval(elastic, (X_train_poly, X_test_poly),(y_train, y_test))

Lambda is: 0.0001
Train set:  MSE=4.885168469233903 R2=0.9408373933787827
Test set:  MSE=17.536105176623664 R2=0.8023485299425339
Lambda is: 0.001
Train set:  MSE=5.022774408558441 R2=0.9391708948520129
Test set:  MSE=17.0500135999814 R2=0.8078273243349152
Lambda is: 0.01
Train set:  MSE=6.258270634244153 R2=0.9242082220920895
Test set:  MSE=17.272043729307587 R2=0.8053247970623901
Lambda is: 0.1
Train set:  MSE=11.45739562799781 R2=0.8612433952458572
Test set:  MSE=25.640899840151665 R2=0.7109984517110435
Lambda is: 1
Train set:  MSE=18.966995179357113 R2=0.770297200260362
Test set:  MSE=35.51343150683474 R2=0.5997240052216382
Lambda is: 10
Train set:  MSE=67.75245698455062 R2=0.17947313681353638
Test set:  MSE=74.68499614194536 R2=0.15821676877431756


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.799e+02, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.537e+02, tolerance: 2.923e+00
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.820e+02, tolerance: 2.923e

Utilizzando ElasticNet, e quindi entrambe le regolarizzazioni, abbiamo ottenuto un modello ancora migliore, con un R2 di 0.81 sul test set e 0.92 sul test set.<br>
Abbiamo il nostro vincitore!

## Che differenza c'è tra la regolarizzazione L2 ed L1 ?

La differenza principale tra le due tecniche di regolarizzazione viste è la seguente:
* La regolarizzazione L2 riduce la magnitudine dei pesi a valori più bassi.
* La regolarizzazione L1 elimina le feature più deboli portando il loro peso a 0.
Nella pratica la L2 porta quasi sempre a migliori risultati, ma utilizzarle entrambe con ElasticNet è anche un ottima soluzione.